# Monte-Carlo AMSE for Four Classical Donoho–Johnstone Test Signals

This notebook performs a unified Monte-Carlo experiment across the four benchmark signals:

- **Doppler**
- **Blocks**
- **HeaviSine**
- **Bumps**

Each signal is sampled at  
$$
N = 1024
$$
and corrupted with Gaussian noise $\varepsilon \sim N(0,\sigma^2)$ scaled so that the **signal-to-noise variance ratio** is
$$
\mathrm{SNR} = \frac{\mathrm{Var}(\text{signal})}{\mathrm{Var}(\text{noise})} = 7.
$$

The **universal wavelet threshold** used for Hard / Soft shrinkage is  
$$
\lambda_U = \sigma \sqrt{2 \log N}.
$$

Shrinkage rules compared:
- Universal **Hard** thresholding
- Universal **Soft** thresholding
- **SCAD** shrinkage (Fan–Li), $\lambda^\*$ chosen **oracle-wise** to minimize true MSE
- **Smooth-SCAD** shrinkage (raised-cosine generator), $\lambda^\*$ oracle-selected

For each signal, we run
$$
M = 1000
$$
Monte-Carlo replications. For replication $m$:
$$
\text{MSE}^{(m)} = \frac{1}{N} \sum_{i=1}^{N} \left(\hat{f}^{(m)}(i) - f(i)\right)^2
$$

The **average MSE (AMSE)** is
$$
\mathrm{AMSE} = \frac{1}{M} \sum_{m=1}^{M} \text{MSE}^{(m)},
$$
and the **standard deviation of MSE** is reported as
$$
\mathrm{Std}(\text{MSE}) = 
\sqrt{\frac{1}{M-1}\sum_{m=1}^{M}
\left( \text{MSE}^{(m)} - \mathrm{AMSE} \right)^2 }.
$$

After running, the script prints a **LaTeX table**:

```
% LaTeX table: AMSE and Std(MSE)
\begin{table}
...
\end{table}
```

This output is suitable for direct inclusion in a manuscript or appendix summarizing numerical simulation results across wavelet shrinkage rules.


In [2]:
import numpy as np
import pywt
import argparse

# ============================================================
# Helper: MSE
# ============================================================
def mse(y_hat, y_true):
    return np.mean((y_hat - y_true) ** 2)


# ============================================================
# SCAD and Smooth SCAD shrinkage
# ============================================================
def scad_threshold(coeffs, lam, a=3.7):
    """
    Classical SCAD shrinkage (Fan & Li).
    coeffs: 1D numpy array of coefficients
    lam:    main threshold
    a:      SCAD shape parameter (> 2)
    """
    out = np.zeros_like(coeffs)
    for i, d in enumerate(coeffs):
        ad = abs(d)
        if ad <= lam:
            out[i] = 0.0
        elif ad <= 2 * lam:
            out[i] = np.sign(d) * (ad - lam)
        elif ad <= a * lam:
            out[i] = np.sign(d) * (((a - 1) * ad - a * lam) / (a - 2))
        else:
            out[i] = d
    return out


def smooth_scad_threshold(coeffs, lam, a=3.0):
    """
    Smooth SCAD shrinkage using raised-cosine generator.
    coeffs: 1D numpy array of coefficients
    lam:    main threshold
    a:      shape parameter (> 1)
    """
    out = np.zeros_like(coeffs)
    for i, d in enumerate(coeffs):
        absd = abs(d)
        if absd <= lam:
            out[i] = 0.0
        elif absd >= a * lam:
            out[i] = d
        else:
            # 0 < (absd - lam)/((a-1) * lam) < 1
            s = (absd - lam) / ((a - 1) * lam)
            h = lam * (np.cos((np.pi / 2.0) * s) ** 2)
            out[i] = np.sign(d) * (absd - h)
    return out


# ============================================================
# Donoho–Johnstone test signals
# ============================================================
def doppler(N):
    x = np.linspace(0, 1, N)
    # Standard Doppler test function
    f = np.sqrt(x * (1 - x)) * np.sin((2 * np.pi * 1.05) / (x + 0.05))
    return f


def blocks(N):
    x = np.linspace(0, 1, N)
    t = np.array([0.1, 0.13, 0.15, 0.23, 0.25,
                  0.40, 0.44, 0.65, 0.76, 0.78,
                  0.81])
    h = np.array([4, -5, 3, -4, 5,
                  -4.2, 2.1, 4.3, -3.1, 2.1,
                  -4.2])
    f = np.zeros_like(x)
    for tk, hk in zip(t, h):
        f += hk * (x > tk)
    return f


def heavisine(N):
    x = np.linspace(0, 1, N)
    f = 4.0 * np.sin(4.0 * np.pi * x) \
        - np.sign(x - 0.3) \
        - np.sign(0.72 - x)
    return f


def bumps(N):
    x = np.linspace(0, 1, N)
    t = np.array([0.1, 0.13, 0.15, 0.23, 0.25,
                  0.40, 0.44, 0.65, 0.76, 0.78,
                  0.81, 0.84, 0.85])
    h = np.array([4.0, 5.0, 3.0, 4.0, 5.0,
                  4.2, 2.1, 4.3, 3.1, 5.1,
                  4.2, 3.3, 5.3])
    w = np.array([0.005, 0.005, 0.006, 0.006, 0.005,
                  0.010, 0.010, 0.010, 0.005, 0.008,
                  0.005, 0.008, 0.005])

    f = np.zeros_like(x)
    for tk, hk, wk in zip(t, h, w):
        f += hk / (1.0 + np.abs((x - tk) / wk) ** 4)
    return f


# ============================================================
# Universal threshold
# ============================================================
def univ_thresh(sigma, N):
    return sigma * np.sqrt(2.0 * np.log(N))


# ============================================================
# Monte Carlo driver for one signal
# ============================================================
def run_mc_signal(
    signal_name,
    generator,
    wavelet,
    N=1024,
    SNR=7.0,
    sigma=1.0,
    M=1000,
    seed=None
):
    """
    Monte Carlo experiment for a single signal.

    Returns: (AMSE_vector, STD_vector)
        each is length 4: [Hard, Soft, SCAD, SmoothSCAD]
    """
    if seed is not None:
        np.random.seed(seed)

    base = generator(N)
    var_base = np.var(base)
    scale = np.sqrt(SNR * sigma**2 / var_base)
    f = base * scale

    MSE_hard = []
    MSE_soft = []
    MSE_scad = []
    MSE_sscad = []

    for _ in range(M):
        noise = sigma * np.random.randn(N)
        y = f + noise

        # Wavelet decomposition
        coeffs = pywt.wavedec(y, wavelet, mode='per')
        cA = coeffs[0]
        dcoeffs = coeffs[1:]

        lam_u = univ_thresh(sigma, N)

        # Hard universal
        d_hard = [pywt.threshold(c, lam_u, mode='hard') for c in dcoeffs]
        rec_hard = pywt.waverec([cA] + d_hard, wavelet, mode='per')
        MSE_hard.append(mse(rec_hard, f))

        # Soft universal
        d_soft = [pywt.threshold(c, lam_u, mode='soft') for c in dcoeffs]
        rec_soft = pywt.waverec([cA] + d_soft, wavelet, mode='per')
        MSE_soft.append(mse(rec_soft, f))

        # SCAD: oracle over lambda grid
        lam_grid = np.linspace(0.1 * lam_u, lam_u, 25)
        best_scad = np.inf
        for lam in lam_grid:
            d_scad = [scad_threshold(c, lam, a=3.7) for c in dcoeffs]
            rec_scad = pywt.waverec([cA] + d_scad, wavelet, mode='per')
            mse_sc = mse(rec_scad, f)
            if mse_sc < best_scad:
                best_scad = mse_sc
        MSE_scad.append(best_scad)

        # Smooth SCAD: oracle over same lambda grid
        best_ss = np.inf
        for lam in lam_grid:
            d_ss = [smooth_scad_threshold(c, lam, a=3.0) for c in dcoeffs]
            rec_ss = pywt.waverec([cA] + d_ss, wavelet, mode='per')
            mse_ss = mse(rec_ss, f)
            if mse_ss < best_ss:
                best_ss = mse_ss
        MSE_sscad.append(best_ss)

    MSE_hard = np.array(MSE_hard)
    MSE_soft = np.array(MSE_soft)
    MSE_scad = np.array(MSE_scad)
    MSE_sscad = np.array(MSE_sscad)

    AMSE = np.array([
        MSE_hard.mean(),
        MSE_soft.mean(),
        MSE_scad.mean(),
        MSE_sscad.mean()
    ])

    STD = np.array([
        MSE_hard.std(ddof=1),
        MSE_soft.std(ddof=1),
        MSE_scad.std(ddof=1),
        MSE_sscad.std(ddof=1)
    ])

    return AMSE, STD


# ============================================================
# Main: run all four signals and print LaTeX table
# ============================================================
def main():
    parser = argparse.ArgumentParser(
        description="Monte Carlo AMSE / Std(MSE) for four test signals "
                    "with Hard, Soft, SCAD, Smooth SCAD."
    )
    parser.add_argument("--N", type=int, default=1024,
                        help="Signal length (default: 1024)")
    parser.add_argument("--snr", type=float, default=7.0,
                        help="SNR in terms of variance ratio (default: 7)")
    parser.add_argument("--M", type=int, default=1000,
                        help="Number of Monte Carlo replications (default: 1000)")
    parser.add_argument("--sigma", type=float, default=1.0,
                        help="Noise standard deviation (default: 1)")
    parser.add_argument("--seed", type=int, default=None,
                        help="Random seed (default: None)")
    args, unknown = parser.parse_known_args()
    N = args.N
    SNR = args.snr
    M = args.M
    sigma = args.sigma
    seed = args.seed

    signals = [
        ("Doppler",   doppler,   "sym4"),
        ("Blocks",    blocks,    "haar"),
        ("HeaviSine", heavisine, "sym4"),
        ("Bumps",     bumps,     "db3"),
    ]

    results = {}

    for name, gen, wav in signals:
        print(f"Running {name}: N={N}, wavelet={wav}, SNR={SNR}, "
              f"sigma={sigma}, M={M}")
        AMSE, STD = run_mc_signal(
            name,
            gen,
            wav,
            N=N,
            SNR=SNR,
            sigma=sigma,
            M=M,
            seed=seed
        )
        results[name] = (AMSE, STD)

    # Print LaTeX table
    print("\n% LaTeX table: AMSE and Std(MSE) for four signals")
    print("\\begin{table}[h!]")
    print("\\centering")
    print("\\caption{AMSE and standard deviation of MSE over "
          f"$M={M}$ runs, $N={N}$, SNR (variance ratio) = {SNR}, "
          "$\\sigma=1$.}")
    print("\\label{tab:amse_std_four_signals}")
    print("\\begin{tabular}{lcccc}")
    print("\\hline")
    print("Signal & Hard & Soft & SCAD & Smooth SCAD \\\\")
    print("\\hline")

    for name, _, _ in signals:
        AMSE, STD = results[name]
        # AMSE row
        print(f"{name} & "
              f"{AMSE[0]:.4e} & {AMSE[1]:.4e} & {AMSE[2]:.4e} & {AMSE[3]:.4e} \\\\")
        # Std(MSE) row
        print(f" & "
              f"({STD[0]:.4e}) & ({STD[1]:.4e}) & ({STD[2]:.4e}) & ({STD[3]:.4e}) \\\\")
        print("\\hline")

    print("\\end{tabular}")
    print("\\end{table}")


if __name__ == "__main__":
    main()


Running Doppler: N=1024, wavelet=sym4, SNR=7.0, sigma=1.0, M=1000
Running Blocks: N=1024, wavelet=haar, SNR=7.0, sigma=1.0, M=1000
Running HeaviSine: N=1024, wavelet=sym4, SNR=7.0, sigma=1.0, M=1000
Running Bumps: N=1024, wavelet=db3, SNR=7.0, sigma=1.0, M=1000

% LaTeX table: AMSE and Std(MSE) for four signals
\begin{table}[h!]
\centering
\caption{AMSE and standard deviation of MSE over $M=1000$ runs, $N=1024$, SNR (variance ratio) = 7.0, $\sigma=1$.}
\label{tab:amse_std_four_signals}
\begin{tabular}{lcccc}
\hline
Signal & Hard & Soft & SCAD & Smooth SCAD \\
\hline
Doppler & 1.5471e-01 & 4.0384e-01 & 1.3499e-01 & 1.3166e-01 \\
 & (1.7342e-02) & (3.1582e-02) & (1.4036e-02) & (1.4280e-02) \\
\hline
Blocks & 2.0132e-01 & 6.6986e-01 & 1.5306e-01 & 1.4498e-01 \\
 & (2.8849e-02) & (4.1063e-02) & (1.9076e-02) & (1.9866e-02) \\
\hline
HeaviSine & 7.7754e-02 & 1.0460e-01 & 6.8019e-02 & 6.5857e-02 \\
 & (1.4550e-02) & (1.1022e-02) & (9.6688e-03) & (1.0148e-02) \\
\hline
Bumps & 3.0011e-01 & 9.6